In [23]:
test_input_1 = '''...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
'''
test_output_1 = 374
test_output_2 = 1030
test_output_3 = 8410

In [2]:
import numpy as np

In [3]:
def parse_input(text):
    return np.array([[c for c in line] for line in text.strip().split('\n')])
grid = parse_input(test_input_1)
grid, grid.shape

(array([['.', '.', '.', '#', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
        ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
        ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
        ['#', '.', '.', '.', '#', '.', '.', '.', '.', '.']], dtype='<U1'),
 (10, 10))

In [4]:
def expand_cosmos(grid):
    for y in range(grid.shape[0]-1,-1,-1):
        if np.all(grid[y,:] == '.'):
            grid=np.vstack((grid[:y+1, :], grid[y,:], grid[y+1:,:]))
    for x in range(grid.shape[1]-1, -1, -1):
        if np.all(grid[:,x] == '.'):
            grid=np.hstack((grid[:, :x+1], np.reshape(grid[:,x], (-1,1)), grid[:,x+1:]))
    return grid
grid = expand_cosmos(grid)
grid

array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
       ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['#', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.']],
      dtype='<U1')

In [5]:
def locate_galaxies(grid):
    ys,xs = np.where(grid == '#')
    return list(zip(ys,xs))
locations = locate_galaxies(grid)
locations

[(0, 4), (1, 9), (2, 0), (5, 8), (6, 1), (7, 12), (10, 9), (11, 0), (11, 5)]

In [7]:
def pair_distances(locations):
    distances = {}
    for i,a in enumerate(locations):
        for b in locations[i+1:]:
            distances[(a,b)] = abs(a[0] - b[0]) + abs(a[1]-b[1])
    return distances
pair_distances(locations)

{((0, 4), (1, 9)): 6,
 ((0, 4), (2, 0)): 6,
 ((0, 4), (5, 8)): 9,
 ((0, 4), (6, 1)): 9,
 ((0, 4), (7, 12)): 15,
 ((0, 4), (10, 9)): 15,
 ((0, 4), (11, 0)): 15,
 ((0, 4), (11, 5)): 12,
 ((1, 9), (2, 0)): 10,
 ((1, 9), (5, 8)): 5,
 ((1, 9), (6, 1)): 13,
 ((1, 9), (7, 12)): 9,
 ((1, 9), (10, 9)): 9,
 ((1, 9), (11, 0)): 19,
 ((1, 9), (11, 5)): 14,
 ((2, 0), (5, 8)): 11,
 ((2, 0), (6, 1)): 5,
 ((2, 0), (7, 12)): 17,
 ((2, 0), (10, 9)): 17,
 ((2, 0), (11, 0)): 9,
 ((2, 0), (11, 5)): 14,
 ((5, 8), (6, 1)): 8,
 ((5, 8), (7, 12)): 6,
 ((5, 8), (10, 9)): 6,
 ((5, 8), (11, 0)): 14,
 ((5, 8), (11, 5)): 9,
 ((6, 1), (7, 12)): 12,
 ((6, 1), (10, 9)): 12,
 ((6, 1), (11, 0)): 6,
 ((6, 1), (11, 5)): 9,
 ((7, 12), (10, 9)): 6,
 ((7, 12), (11, 0)): 16,
 ((7, 12), (11, 5)): 11,
 ((10, 9), (11, 0)): 10,
 ((10, 9), (11, 5)): 5,
 ((11, 0), (11, 5)): 5}

In [10]:
def solve1(text):
    return sum(pair_distances(locate_galaxies(expand_cosmos(parse_input(text)))).values())
assert solve1(test_input_1) == test_output_1

In [11]:
with open('day11.txt') as FILE:
    print(solve1(FILE.read()))

9799681


In [18]:
class ExpCosmos:

    def __init__(self, grid, factor):
        self.grid = grid
        self.exp_factor = factor
        self.expansions = [
            [y for y in range(self.grid.shape[0]) if np.all(grid[y,:] == '.')],
            [x for x in range(self.grid.shape[1]) if np.all(grid[:,x] == '.')],
        ]
        self.__locate_galaxies()
        
    def __locate_galaxies(self):
        ys,xs = np.where(self.grid == '#')
        self.gal_locs = list(zip(ys,xs))

    def distance(self, loc1, loc2):
        dist = [None, None]
        for ax,_ in enumerate(dist):
            source, destination = sorted([loc1[ax], loc2[ax]])
            dist[ax] = destination - source
            for exp in self.expansions[ax]:
                if source < exp < destination:
                    dist[ax] -= 1
                    dist[ax] += self.exp_factor
        return sum(dist)

    def __repr__(self):
        return f"{self.exp_factor}\n{self.expansions}\n{self.gal_locs}"

In [26]:
def solve2(text, factor=1_000_000):
    grid = parse_input(text)
    cosmos = ExpCosmos(grid, factor)
    distances = {}
    for i,a in enumerate(cosmos.gal_locs):
        for b in cosmos.gal_locs[i+1:]:
            distances[(a,b)] = cosmos.distance(a,b)
    return sum(distances.values())
assert solve2(test_input_1, 2) == test_output_1
assert solve2(test_input_1, 10) == test_output_2
assert solve2(test_input_1, 100) == test_output_3

In [27]:
with open('day11.txt') as FILE:
    print(solve2(FILE.read()))

513171773355
